# Refactor Notebook: Use Context Dictionary for State Management

This notebook has been refactored to use a single `context` dictionary for all state management. Each step stores and retrieves state from `context`, ensuring consistent and explicit handling of variables throughout the workflow.

## 1. Refactor Setup and Authentication to Store State in Context

The authentication logic below ensures that the GIS connection and all related state are stored in the `context` dictionary (e.g., `context['gis']`). All subsequent cells will reference `context` for state.

In [1]:
import ipywidgets as widgets
from arcgis.gis import GIS
import sys
import warnings
from bs4 import MarkupResemblesLocatorWarning

warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# Initialize context dictionary for all state
context = {
    "gis": None,
    "classic_item": None,
    "classic_item_data": None,
    "classic_story_title": None,
    "classic_story_subtitle": None,
    "classic_story_type": None,
    "classic_story_panel_position": None,
    "classic_story_theme": None,
    "new_theme": None,
    "entries": [],
    "entry_titles": [],
    "main_stage_contents": [],
    "invalid_webmaps": [],
    "published_storymap_items": [],
    "thumbnail_paths": [],
    "collection_title": None,
    "collection_url": None,
    "collection_id": None,
    "folder_name": None,
}

def initialize_ui(widget_type, description="", width=300, height=40, value=None, layout=None, options=None):
    import ipywidgets as widgets
    if not layout:
        layout = widgets.Layout(width=f"{width}px", height=f"{height}px")
    if widget_type == "button":
        return widgets.Button(description=description, layout=layout)
    elif widget_type == "checkbox":
        return widgets.Checkbox(value=value if value is not None else False, description=description, layout=layout)
    elif widget_type == "text":
        return widgets.Text(value=value if value is not None else "", description=description, layout=layout)
    elif widget_type == "label":
        return widgets.Label(value=value if value is not None else "", layout=layout)
    elif widget_type == "output":
        return widgets.Output()
    elif widget_type == "hbox":
        return widgets.HBox(options if options else [])
    else:
        raise ValueError("Unsupported widget_type")

output1 = initialize_ui("output")
input1 = initialize_ui("checkbox", value=True, description="Yes/No")
txt1 = initialize_ui("label", value="Are you running this within ArcGIS Online?")
btn1 = initialize_ui("button", description="Setup Notebook", width="200px")
HBox1 = initialize_ui("hbox", options=[txt1, input1])

def setup_notebook(button):
    with output1:
        output1.clear_output()
        print("Setting up the notebook environment...")
        print(f"\tPython version: {sys.version}")
        try:
            import arcgis
            print(f"\tArcGIS for Python API version: {arcgis.__version__}")
        except ImportError:
            print("ArcGIS for Python API not installed.")
        agoNotebook = input1.value
        if agoNotebook == False:
            username_widget = widgets.Text(
                value='',
                placeholder='Enter your ArcGIS Online username',
                layout=widgets.Layout(width='400px')
            )
            submit_button = widgets.Button(description="Login")
            auth_Hbox = widgets.HBox([username_widget, submit_button])
            output_auth = widgets.Output()
            display(auth_Hbox, output_auth)

            def handle_login(button):
                with output_auth:
                    output_auth.clear_output()
                    print("Logging in...")
                    try:
                        import keyring
                        service_name = "system"
                        username_for_keyring = username_widget.value
                        credential = keyring.get_credential(service_name, username_for_keyring)
                        if credential is None:
                            print(f"'{username_for_keyring}' is not in the local system's credential store. Try another username.")
                        else:
                            password_from_keyring = keyring.get_password("system", username_for_keyring)
                            portal_url = 'https://www.arcgis.com'
                            context['gis'] = GIS(portal_url, username=username_for_keyring, password=password_from_keyring)
                            print(f"\tSuccessfully logged in as: {context['gis'].properties.user.username}")
                            print("\nStep #1 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")
                    except ImportError:
                        print("The 'keyring' module is not installed. Please install it using 'pip install keyring'.")
            submit_button.on_click(handle_login)
        else:
            context["gis"] = GIS("home")
            print(f"\tSuccessfully logged in as: {context['gis'].properties.user.username}")
            print("\nStep #1 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")

display(widgets.VBox([HBox1, btn1, output1]))
btn1.on_click(setup_notebook)

## 2. Update Data Fetching to Use Context for Item and Data

The data fetching logic below stores the classic StoryMap item and its data in `context['classic_item']` and `context['classic_item_data']`. All downstream logic references these context keys.

In [ ]:
from arcgis.gis import Item

output2 = initialize_ui("output")
txt2 = initialize_ui("label", value="Paste 32-digit Classic Esri Story Map id -->")
input2 = initialize_ui("text", description="Item ID:", width=400)
HBox2 = initialize_ui("hbox", options=[txt2, input2])
btn2 = initialize_ui("button", description="Fetch Story Data", width="200px")

def safe_get_json(item):
    try:
        data = item.get_data()
        if isinstance(data, dict):
            return data
        elif isinstance(data, str):
            import json
            return json.loads(data)
        else:
            return {}
    except Exception:
        return {}

def fetch_classic_storymap_data(classic_storymap_id, context):
    gis = context["gis"]
    classic_item = Item(gis=gis, itemid=classic_storymap_id)
    classic_item_data = safe_get_json(classic_item)
    if classic_item_data == {}:
        raise ValueError("ERROR: StoryMap to be converted must be hosted on ArcGIS Online.")
    context["classic_item"] = classic_item
    context["classic_item_data"] = classic_item_data

def fetch_story_data(button):
    with output2:
        output2.clear_output()
        print("Fetching data...")
        classic_storymap_id = input2.value
        try:
            fetch_classic_storymap_data(classic_storymap_id, context)
            if context["classic_item"] is not None and context["classic_item_data"] is not None:
                print(f"Fetched classic StoryMap: '{context['classic_item'].title}' (ID: {context['classic_item'].itemid})")
            else:
                print("Could not fetch classic StoryMap data. Check the item ID and try again.")
        except Exception as e:
            print(f"Error: {e}")
        print("\nStep #2 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")

display(widgets.VBox([HBox2, btn2, output2]))
btn2.on_click(fetch_story_data)

## 3. Modify Settings and Theme Extraction to Store in Context

Settings, theme, and entries are now extracted and stored in the context dictionary (e.g., `context['classic_story_title']`, `context['entries']`, etc.).

In [ ]:
output3 = widgets.Output()
user_button3 = widgets.Button(description="Extract Settings")

def extract_story_settings(classic_item_data, context):
    values = classic_item_data.get("values", {})
    context["classic_story_title"] = values.get("title", "Untitled Classic StoryMap Series").strip()
    context["classic_story_subtitle"] = values.get("subtitle", "")
    settings = values.get("settings", {})
    context["classic_story_type"] = settings.get("layout", {}).get("id", "Unknown")
    context["classic_story_panel_position"] = settings.get("layoutOptions", {}).get("panel", {}).get("position", "Unknown")
    context["classic_story_theme"] = settings.get("theme", {})
    context["entries"] = values.get("story", {}).get("entries", [])
    return context

def determine_theme(theme):
    from arcgis.apps.storymap import Themes
    classic_name = theme.get("colors", {}).get("name", "No classic theme name")
    group = theme.get("colors", {}).get("group", "light")
    if group == "dark":
        return classic_name, Themes.OBSIDIAN
    else:
        return classic_name, Themes.SUMMIT

def extract_and_display_settings(button):
    with output3:
        output3.clear_output()
        if context["classic_item_data"] is None:
            print("No classic StoryMap data found. Fetch the data first.")
            return
        extract_story_settings(context["classic_item_data"], context)
        entries = context["entries"]
        print("\nStory settings:")
        print(f"{'panel position:':>15} {context['classic_story_panel_position']}")
        print(f"{'series title:':>15} '{context['classic_story_title']}'")
        if context["classic_story_subtitle"]:
            print(f"{'subtitle:':>15} {context['classic_story_subtitle']}")
        print(f"{'series type:':>15} {context['classic_story_type']}")
        print(f"\nFound {len(entries)} entries in the Classic Map Series.")
        for i, e in enumerate(entries):
            print(f"{i+1}. {e['title']}")
        classic_name, new_theme = determine_theme(context["classic_story_theme"])
        context["new_theme"] = new_theme
        print(f"\nClassic theme name: {classic_name}")
        print(f"{'New theme set to:':>19} {new_theme.name}")
        print("\nStep #3 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")

display(user_button3)
display(output3)
user_button3.on_click(extract_and_display_settings)

Button(description='Extract Settings', style=ButtonStyle())

Output()

## 4. Process Entries and Store Results in Context

Entry processing now stores entry titles, main stage contents, and invalid webmaps in context (e.g., `context['entry_titles']`, `context['main_stage_contents']`, `context['invalid_webmaps']`).

In [ ]:
output4 = widgets.Output()
input_param4 = widgets.Button(description="Process Entries")

def process_entry(entry, entry_idx, entries, story_settings, gis):
    from arcgis.apps.storymap import Map, Embed, Image
    entry_title = entry.get("title")
    media_info = entry.get("media", {})
    media_type = media_info.get("type")
    main_stage_content = None
    invalid_webmap = False
    if media_type == "webmap":
        webmap_id = media_info.get('webmap', {}).get('id')
        if webmap_id and not invalid_webmap:
            try:
                main_stage_content = Map(webmap_id)
            except Exception as e:
                print(f"Error creating Map object for {entry_title} ({webmap_id}): {e}")
                invalid_webmap = True
    elif media_type == "webpage":
        webpage_url = media_info.get("webpage", {}).get("url")
        if webpage_url:
            main_stage_content = Embed(webpage_url)
    elif media_type == "image":
        image_url = media_info.get("image", {}).get("url")
        if image_url:
            main_stage_content = Image(image_url)
    return entry_title, main_stage_content, invalid_webmap

def process_all_entries(entries, context):
    gis = context["gis"]
    story_settings = context["classic_item_data"].get("values", {}).get("settings", {})
    context["entry_titles"] = [None] * len(entries)
    context["main_stage_contents"] = [None] * len(entries)
    context["invalid_webmaps"] = [False] * len(entries)
    for i, entry in enumerate(entries):
        title, content, invalid = process_entry(entry, i, entries, story_settings, gis)
        context["entry_titles"][i] = title
        context["main_stage_contents"][i] = content
        context["invalid_webmaps"][i] = invalid
        if invalid:
            print(f"WARNING: There is a problem with the webmap in entry [{i+1} of {len(entries)}]: {title}. Please fix before publishing the new StoryMap.")
        print(f"[{i+1} of {len(entries)}]: {title:35} Media type: {type(content).__name__}")

def process_entries(button):
    with output4:
        output4.clear_output()
        entries = context["entries"]
        print(f"Processing {len(entries)} entries...")
        process_all_entries(entries, context)
        print("\nStep #4 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")

display(input_param4)
display(output4)
input_param4.on_click(process_entries)

Button(description='Process Entries', style=ButtonStyle())

Output()

## 5. Create StoryMaps and Save Items/Thumbnails in Context

StoryMap creation logic now stores published StoryMap items and thumbnail paths in `context['published_storymap_items']` and `context['thumbnail_paths']`.

In [5]:
output5 = widgets.Output()
user_button5 = widgets.Button(description="Create ArcGIS StoryMaps from each entry", layout=widgets.Layout(width='300px', height='40px'))

def build_and_save_storymap(entry, entry_index, entry_title, main_stage_content, new_theme, default_thumbnail_path, gis):
    from arcgis.apps.storymap import StoryMap, Sidecar, Image
    import tempfile, os
    story = StoryMap()
    story.theme(new_theme)
    sidecar = Sidecar(style="docked-panel")
    story.add(sidecar)
    description_html = entry.get("description", "")
    sidecar.add_slide(contents=[description_html], media=main_stage_content)
    cover_properties = story.content_list[0]
    cover_properties.title = entry_title
    cover_properties.byline = ""
    cover_properties.date = "none"
    cover_properties.media = Image(default_thumbnail_path)
    for k, v in story.properties['nodes'].items():
        if v['type'] == 'storycover':
            v['config'] = {'isHidden': 'true'}
    story.save(title=entry_title, tags=["Classic Story Map to AGSM Conversion", "Story Map Series"], publish=True)
    published_story_item = getattr(story, '_item', None)
    thumbnail_path = default_thumbnail_path
    if published_story_item:
        published_story_item.update(thumbnail=thumbnail_path)
        published_story_item_url = "https://storymaps.arcgis.com/stories/" + published_story_item.id
        print(f"{published_story_item_url} '{entry_title}' is staged for publishing. Click the link to complete.")
        return story, published_story_item, thumbnail_path
    else:
        print("Could not find item for story:", story.title)
        return story, None, thumbnail_path

def create_and_save_storymaps(entries, context):
    gis = context["gis"]
    new_theme = context["new_theme"]
    default_thumbnail_path = "https://cdn-a.arcgis.com/cdn/1BE082D/js/arcgis-app-components/arcgis-app/assets/arcgis-item-thumbnail/storymap.png"
    context["published_storymap_items"] = [None] * len(entries)
    context["thumbnail_paths"] = [None] * len(entries)
    print("\n***NOTICE*** You MUST click each link below to open the story in a browser tab. ***NOTICE***\n")
    for i, entry in enumerate(entries):
        print(f"[{i+1} of {len(entries)}]... ", end="")
        story, published_story_item, thumbnail_path = build_and_save_storymap(
            entry, i, entry["title"], context["main_stage_contents"][i], new_theme, default_thumbnail_path, gis
        )
        if published_story_item:
            context["published_storymap_items"][i] = published_story_item
        context["thumbnail_paths"][i] = thumbnail_path

def create_storymaps(button):
    with output5:
        output5.clear_output()
        entries = context["entries"]
        print(f"Creating and saving {len(entries)} StoryMaps...")
        create_and_save_storymaps(entries, context)
        print("\nStep #5 complete. Ensure you have opened each story and checked for errors before continuing. \nOnce all stories have been successfully published, Click the Markdown text below and then click the 'Play' button twice to proceed.")

display(user_button5)
display(output5)
user_button5.on_click(create_storymaps)

Button(description='Create ArcGIS StoryMaps from each entry', layout=Layout(height='40px', width='300px'), sty…

Output()

## 6. Build Collection and Store Metadata in Context

Collection creation now stores collection title, URL, and ID in context (e.g., `context['collection_title']`, `context['collection_url']`, `context['collection_id']`).

In [ ]:
output6 = widgets.Output()
user_button6 = widgets.Button(description="Create Collection", layout=widgets.Layout(width='150px', height='40px'))

def build_collection(context):
    from arcgis.apps.storymap import Collection, Image
    classic_item = context["classic_item"]
    published_storymap_items = context["published_storymap_items"]
    thumbnail_paths = context["thumbnail_paths"]
    classic_story_type = context["classic_story_type"]
    new_theme = context["new_theme"]
    collection = Collection()
    collection_title = classic_item.title
    for i, story in enumerate(published_storymap_items):
        if story is None:
            print(f"Story {i+1} is None. Skipping.")
            continue
        try:
            collection.add(item=story, title=story.title, thumbnail=thumbnail_paths[i])
        except Exception as e:
            print(f"Error adding story '{story.title}' to Collection: {e}")
    collection.content[0].title = collection_title
    collection.content[0].byline = ""
    collection.theme(new_theme)
    if classic_story_type == "accordion":
        collection.content[1].type = "tab"
    else:
        collection.content[1].type = classic_story_type
    collection.content[1].media = Image(thumbnail_paths[0])
    published_collection = collection.save(title=collection_title, tags=["Classic Story Map to AGSM Conversion", "Story Map Series"], publish=True)
    context["collection_title"] = collection_title
    context["collection_url"] = collection._url
    context["collection_id"] = published_collection.id

def create_collection(button):
    with output6:
        output6.clear_output()
        print(f"Creating Collection '{context['classic_story_title']}'...")
        build_collection(context)
        print(f"Collection staged: '{context['collection_title']}' {context['collection_url']} \nClick the link to open the Collection builder. Make any desired edits and then complete the publication of your converted StoryMap.")
        print("\nStep #6 complete. Once you've published the Collection, click the Markdown text below and then click the 'Play' button twice to proceed.")

display(user_button6)
display(output6)
user_button6.on_click(create_collection)

## 7. Folder Creation and Management Using Context

Folder creation/check logic now stores folder name and related state in `context['folder_name']`.

In [ ]:
output7 = widgets.Output()
input_param7 = widgets.Text(value="", description="", layout=widgets.Layout(width='800px'))
user_button7 = widgets.Button(description="Check for folder")
user_button7_1 = widgets.Button(description="Create folder")

def check_folder(button):
    with output7:
        output7.clear_output()
        if not context["classic_story_title"]:
            print("No classic StoryMap title found. Extract the story settings first.")
            return
        folder_name = "Collection-" + context["classic_story_title"] if context["classic_story_title"] else "Collection-Conversion"
        input_param7.value = folder_name
        context["folder_name"] = folder_name
        user_line7 = widgets.HBox([widgets.Label(value="Edit the folder name if desired -->"), input_param7])
        user = context["gis"].users.me
        existing_folders = context["gis"].content.folders.list(user.username)
        folder_names = [f.name for f in existing_folders]
        if folder_name in folder_names:
            print(f"Folder '{folder_name}' already exists. Saving results there.")
            print("\nStep #7 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")
        else:
            display(user_line7)
            display(user_button7_1)

def create_folder(button):
    with output7:
        output7.clear_output()
        folder_name = input_param7.value.strip() if input_param7.value.strip() else context["folder_name"]
        context["folder_name"] = folder_name
        try:
            context["gis"].content.folders.create(folder=folder_name, owner=context["gis"].users.me.username)
            print(f"Created folder '{folder_name}' to save entries and Collection.")
        except Exception as e:
            print(f"Error creating folder '{folder_name}': {e}")
        print("\nStep #7 complete. Click the Markdown text below and then click the 'Play' button twice to proceed.")

user_button7.on_click(check_folder)
user_button7_1.on_click(create_folder)
display(user_button7)
display(output7)

## 8. Move Items to Folder Using Context

Item moving logic now uses context for accessing GIS, items, folder name, and collection ID.

In [ ]:
output8 = widgets.Output()
user_button8 = widgets.Button(description="Move all items to folder", layout=widgets.Layout(width='200px', height='40px'))

def move_item_to_folder(gis, item, folder_name):
    try:
        if item.owner == gis.users.me.username:
            item.move(folder_name)
            print(f"Moved item '{item.title}' (ID: {item.itemid}) to folder '{folder_name}'.")
    except Exception as e:
        print(f"Error moving item '{item.title}' (ID: {item.itemid}): {e}")

def move_items_to_folder(button):
    with output8:
        output8.clear_output()
        folder_name = context["folder_name"]
        gis = context["gis"]
        print(f"Moving items to folder '{folder_name}'...")
        for story_item in context["published_storymap_items"]:
            if story_item:
                move_item_to_folder(gis, story_item, folder_name)
        try:
            if context["collection_id"]:
                collection_item = gis.content.get(context["collection_id"])
                move_item_to_folder(gis, collection_item, folder_name)
                print(f"Moved collection '{context['collection_title']}' to folder '{folder_name}'.")
            else:
                print(f"Could not find the collection item '{context['collection_title']}' to move.")
        except Exception as e:
            print(f"Error moving collection item: {e}")
        print("\nStep #8 complete. Conversion complete!")

user_button8.on_click(move_items_to_folder)
display(user_button8)
display(output8)